###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es la Fiscalía General de la Nación

In [3]:
import pandas as pd

###Archivo 4mnf-va5w.csv

Contiene el total de víctimas según las entradas de noticias criminales por delito al Sistema Penal Oral Acusatorio en la Ley 906 de 2004 y Ley 1098 de 2006 desde hechos ocurridos en 2010

In [4]:
df = pd.read_csv("https://www.datos.gov.co/resource/4mnf-va5w.csv?$limit=4328038")
df.head(3)

<ipython-input-4-dd3b6213d593>:1: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("https://www.datos.gov.co/resource/4mnf-va5w.csv?$limit=4328038")


,criminalidad,es_archivo,es_preclusion,estado,etapa_caso,ley,pais_hecho,departamento_hecho,municipio_hecho,seccional,...,grupo_delito,victima_consumado,sexo,grupo_etario,pais_nacimiento,aplica_lgbti,aplica_nna,indigena,afrodescendiente,total_victimas
0,NO,SI,NO,INACTIVO,INDAGACIÓN,Ley 906,Colombia,Chocó,QUIBDO,DIRECCIÓN SECCIONAL DE CHOCÓ,...,HOMICIDIO DOLOSO,NO,MASCULINO,SIN DATO,Colombia,NO,NO,NO,NO,2
1,NO,SI,NO,INACTIVO,INDAGACIÓN,Ley 906,Colombia,Caquetá,FLORENCIA,DIRECCIÓN SECCIONAL DE CAQUETÁ,...,HOMICIDIO DOLOSO,NO,MASCULINO,Adulto entre 27 y 59 años.,Colombia,NO,NO,NO,NO,18
2,SI,SI,NO,INACTIVO,INDAGACIÓN,Ley 906,Colombia,"BOGOTÁ, D. C.","BOGOTÁ, D.C.",DIRECCIÓN SECCIONAL DE BOGOTÁ,...,HOMICIDIO DOLOSO,SI,MASCULINO,Adulto entre 27 y 59 años.,Colombia,NO,NO,NO,NO,156


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4328038 entries, 0 to 4328037
Data columns (total 24 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   criminalidad        object
 1   es_archivo          object
 2   es_preclusion       object
 3   estado              object
 4   etapa_caso          object
 5   ley                 object
 6   pais_hecho          object
 7   departamento_hecho  object
 8   municipio_hecho     object
 9   seccional           object
 10  a_o_hechos          int64 
 11  a_o_entrada         object
 12  a_o_denuncia        object
 13  delito              object
 14  grupo_delito        object
 15  victima_consumado   object
 16  sexo                object
 17  grupo_etario        object
 18  pais_nacimiento     object
 19  aplica_lgbti        object
 20  aplica_nna          object
 21  indigena            object
 22  afrodescendiente    object
 23  total_victimas      int64 
dtypes: int64(2), object(22)
memory usage: 792.5+ MB


###- Eliminar victimas de hechos que no hayan ocurrido en Colombia

In [ ]:
victim_count = df[df['pais_hecho'] =='Colombia']
len(victim_count)

4327952

### Eliminación de columnas irrelevantes para el proyecto

In [ ]:
relevant_cols = ['departamento_hecho', 'municipio_hecho','a_o_hechos', 'delito','grupo_delito', 'sexo', 'grupo_etario', 'total_victimas']
victim_count = victim_count[relevant_cols]
victim_count.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4327952 entries, 0 to 4328037
Data columns (total 8 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   departamento_hecho  object
 1   municipio_hecho     object
 2   a_o_hechos          int64 
 3   delito              object
 4   grupo_delito        object
 5   sexo                object
 6   grupo_etario        object
 7   total_victimas      int64 
dtypes: int64(2), object(6)
memory usage: 297.2+ MB


### Convertir columna a_o_hechos a tipo date

In [ ]:
# Convertir la columna 'a_o_hechos' a tipo datetime
victim_count['a_o_hechos'] = pd.to_datetime(victim_count['a_o_hechos'], format='%Y')
victim_count['a_o_hechos'] = victim_count['a_o_hechos'].dt.year

<ipython-input-6-5d89094d9c45>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  victim_count['a_o_hechos'] = pd.to_datetime(victim_count['a_o_hechos'], format='%Y')


### Verificar valores nulos

In [ ]:
victim_count.isnull().sum()

,0
departamento_hecho,0
municipio_hecho,0
a_o_hechos,0
delito,0
grupo_delito,0
sexo,0
grupo_etario,0
total_victimas,0


### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [ ]:
# Imprimir categorías únicas para columnas de tipo object
for column in victim_count.select_dtypes(include='object').columns:
    print(f"Categorías en la columna '{column}':")
    print(victim_count[column].unique())
    print()

Categorías en la columna 'departamento_hecho':
['Chocó' 'Caquetá' 'BOGOTÁ, D. C.' 'Antioquia' 'Valle del Cauca' 'Bolívar'
 'Magdalena' 'Norte de Santander' 'Risaralda' 'Atlántico' 'Cesar'
 'Santander' 'Nariño' 'Boyaca' 'Putumayo' 'Cundinamarca' 'Tolima' 'Meta'
 'Caldas' 'Córdoba' 'Huila' 'Quindío' 'Cauca' 'Casanare' 'La Guajira'
 'Archipiélago de San Andrés, Providencia y Santa Catalina' 'Sucre'
 'Vaupés' 'Amazonas' 'Arauca' 'Vichada' 'Guaviare' 'Guainía']

Categorías en la columna 'municipio_hecho':
['QUIBDO' 'FLORENCIA' 'BOGOTÁ, D.C.' ... 'MORICHAL' 'PANA PANA' 'UBATÉ']

Categorías en la columna 'delito':
['HOMICIDIO ART. 103 C.P.' 'HOMICIDIO ART. 103 C.P. AGRAVADO'
 'HOMICIDIO CULPOSO ART. 109 C.P.' ...
 'DISPOSICION DE BIEN PROPIO GRAVADO CON PRENDA. ART. 255 C.P. MENOR CUANTIA'
 'DE LA EXPLOTACION SEXUAL'
 'DELITOS CONTRA LA EXISTENCIA Y SEGURIDAD DEL ESTADO']

Categorías en la columna 'grupo_delito':
['HOMICIDIO DOLOSO' 'HOMICIDIO CULPOSO' 'DELITOS SEXUALES'
 'VIOLENCIA INTRAFAMI

  - Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [ ]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [ ]:
# Aplicar la función a todas las columnas categóricas
categorical_columns = ['departamento_hecho', 'municipio_hecho', 'delito', 'grupo_delito', 'sexo', 'grupo_etario']

for col in categorical_columns:
    victim_count[col] = victim_count[col].apply(remove_accents_and_special_chars)

 - Codificación de algunas variables categóricas

## Agregar código del municipio a dataframe victim_count para facilitar la georeferenciación

### -  Cargar los datos con códigos de departamentos y municipios

Como producto de una consulta a la base de datos del proyecto que se esta construyendo (Tablas departments y municipalities) se creo el archivo csv que se carga en la siguiente celda, y que incluye los nombres de los departamentos y municipios con sus respectivos codigos, generados por el DANE

In [ ]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


### -  Asegurarse de que los nombres de los departamentos estén escritos de manera identica en cada dataframe

  - Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [ ]:
# Aplicar funcion a columnas 'dept_name', 'mupio_name'
for col in dept_mpios_codes[['dept_name', 'mupio_name']]:
    dept_mpios_codes[col] = dept_mpios_codes[col].apply(remove_accents_and_special_chars)

 - Comparar que los nombres de los departamentos y municipios en cada dataframe esten escritos correctamente

In [ ]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)
    print(f"{label2} que no están en {label1}:")
    print(only_in_list2)
    print()


  - Comparar nombres de departamentos en los dataframes

In [ ]:
# Comparar listas de departamento
compare_lists(victim_count['departamento_hecho'], dept_mpios_codes['dept_name'],
              "Departamentos en victim-count", "Departamentos dept_mpios_codes")

Departamentos en victim-count que no están en Departamentos dept_mpios_codes:
{'BOGOTA D. C.'}
Departamentos dept_mpios_codes que no están en Departamentos en victim-count:
{'BOGOTA D.C.'}



 - Solucionar discrepancias en los nombres de departamentos

In [ ]:
# Diccionario de mapeo basado en los resultados de la comparación departamentos
depto_mapping = { 'BOGOTA D. C.': 'BOGOTA D.C.'}

# Reemplazar los nombres incorrectos  el dataframe victim_count
victim_count['departamento_hecho'] = victim_count['departamento_hecho'].replace(depto_mapping)

 - Confirmar que discrepancia se solucionó

In [ ]:
# Comparar listas combinadas de departamento + municipio
compare_lists(victim_count['departamento_hecho'], dept_mpios_codes['dept_name'],
              "Departamentos en victim_count", "Departamentos dept_mpios_codes")

Departamentos en victim_count que no están en Departamentos dept_mpios_codes:
set()
Departamentos dept_mpios_codes que no están en Departamentos en victim_count:
set()



 - Comparar nombres de los departamentos combinados con los nombres de los municipios en los dos dataframe

In [ ]:
# Crear columnas combinadas de departamento + municipio
victim_count['departamento_municipio'] = victim_count['departamento_hecho'] + ' - ' + victim_count['municipio_hecho']
dept_mpios_codes['departamento_municipio'] = dept_mpios_codes['dept_name'] + ' - ' + dept_mpios_codes['mupio_name']

# Comparar listas combinadas de departamento + municipio
compare_lists(victim_count['departamento_municipio'], dept_mpios_codes['departamento_municipio'],
              "Departamentos y Municipios en victim_count", "Departamentos y Municipios en dept_mpios_codes")

Departamentos y Municipios en victim_count que no están en Departamentos y Municipios en dept_mpios_codes:
{'GUAINIA - BARRANCO MINAS', 'ANTIOQUIA - SAN PEDRO', 'VAUPES - PAPUNAUA', 'ANTIOQUIA - SANTAFE DE ANTIOQUIA', 'CAUCA - LOPEZ', 'CESAR - MANAURE', 'CUNDINAMARCA - UBATE', 'BOLIVAR - CARTAGENA', 'MAGDALENA - CHIBOLO', 'CORDOBA - PURISIMA', 'CAUCA - EL BORDO', 'CAUCA - SOTARA', 'NARINO - TUMACO', 'GUAINIA - MAPIRIPANA', 'BOLIVAR - MOMPOS', 'CHOCO - BELEN DE BAJIRA', 'PUTUMAYO - LEGUIZAMO', 'TOLIMA - MARIQUITA', 'BOYACA - GUICAN', 'SUCRE - TOLU VIEJO', 'ANTIOQUIA - DON MATIAS', 'MAGDALENA - CERRO SAN ANTONIO', 'NORTE DE SANTANDER - CUCUTA', 'CUNDINAMARCA - SAN JUAN DE RIO SECO', 'CORDOBA - SAN ANDRES SOTAVENTO', 'CAUCA - PIENDAMO', 'ANTIOQUIA - SAN VICENTE', 'NARINO - CUASPUD'}
Departamentos y Municipios en dept_mpios_codes que no están en Departamentos y Municipios en victim_count:
{'MAGDALENA - CERRO DE SAN ANTONIO', 'GUAINIA - BARRANCOMINAS', 'ANTIOQUIA - DONMATIAS', 'CAUCA - SOTA

 - Corregir nombres de los departamentos combinados con los nombres en dataframe victim_count

In [ ]:
mapeo_manual = {
    'CUNDINAMARCA - SAN JUAN DE RIO SECO': 'CUNDINAMARCA - SAN JUAN DE RIOSECO',
    'TOLIMA - MARIQUITA': 'TOLIMA - SAN SEBASTIAN DE MARIQUITA',
    'CORDOBA - PURISIMA': 'CORDOBA - PURISIMA DE LA CONCEPCION',
    'BOLIVAR - MOMPOS': 'BOLIVAR - SANTA CRUZ DE MOMPOX',
    'NORTE DE SANTANDER - CUCUTA': 'NORTE DE SANTANDER - SAN JOSE DE CUCUTA',
    'CAUCA - EL BORDO': 'CAUCA - PATIA',
    'NARINO - TUMACO': 'NARINO - SAN ANDRES DE TUMACO',
    'CAUCA - SOTARA': 'CAUCA - SOTARA PAISPAMBA',
    'ANTIOQUIA - SAN PEDRO': 'ANTIOQUIA - SAN PEDRO DE LOS MILAGROS',
    'ANTIOQUIA - DON MATIAS': 'ANTIOQUIA - DONMATIAS',
    'CHOCO - BELEN DE BAJIRA': 'ANTIOQUIA - MUTATA',
    'BOLIVAR - CARTAGENA': 'BOLIVAR - CARTAGENA DE INDIAS',
    'NARINO - CUASPUD': 'NARINO - CUASPUD CARLOSAMA',
    'CORDOBA - SAN ANDRES SOTAVENTO': 'CORDOBA - SAN ANDRES DE SOTAVENTO',
    'MAGDALENA - CERRO SAN ANTONIO': 'MAGDALENA - CERRO DE SAN ANTONIO',
    'BOYACA - GUICAN': 'BOYACA - GUICAN DE LA SIERRA',
    'CUNDINAMARCA - UBATE': 'CUNDINAMARCA - VILLA DE SAN DIEGO DE UBATE',
    'CAUCA - LOPEZ': 'CAUCA - LOPEZ DE MICAY',
    'CESAR - MANAURE': 'CESAR - MANAURE BALCON DEL CESAR',
    'CAUCA - PIENDAMO': 'CAUCA - PIENDAMO - TUNIA',
    'GUAINIA - BARRANCO MINAS': 'GUAINIA - BARRANCOMINAS',
    'MAGDALENA - CHIBOLO': 'MAGDALENA - CHIVOLO',
    'SUCRE - TOLU VIEJO': 'SUCRE - SAN JOSE DE TOLUVIEJO',
    'GUAINIA - MAPIRIPANA': 'GUAINIA - PANA PANA',
    'ANTIOQUIA - SANTAFE DE ANTIOQUIA': 'ANTIOQUIA - SANTA FE DE ANTIOQUIA',
    'ANTIOQUIA - SAN VICENTE': 'ANTIOQUIA - SAN VICENTE FERRER',
    'VAUPES - PAPUNAUA': 'VAUPES - PAPUNAHUA',
    'PUTUMAYO - LEGUIZAMO': 'PUTUMAYO - PUERTO LEGUIZAMO',

}


victim_count['departamento_municipio'] = victim_count['departamento_municipio'].replace(mapeo_manual)

 - Comparar nombres de los departamentos combinados con los nombres de los municipios despues de corregir

In [ ]:
# Comparar listas combinadas de departamento + municipio
compare_lists(victim_count['departamento_municipio'], dept_mpios_codes['departamento_municipio'],
              "Departamentos y Municipios en victim_count", "Departamentos y Municipios en dept_mpios_codes")

Departamentos y Municipios en victim_count que no están en Departamentos y Municipios en dept_mpios_codes:
set()
Departamentos y Municipios en dept_mpios_codes que no están en Departamentos y Municipios en victim_count:
set()



### Hacer merge entre los dataframes victim_count y dept_mpios_codes

In [ ]:
# Realizamos el merge para obtener el código de municipio
victim_count_with_codes = victim_count.merge(dept_mpios_codes,
                                         left_on=['departamento_municipio'],
                                         right_on=['departamento_municipio'],
                                         how='left')

# Verificamos el resultado
victim_count_with_codes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4327952 entries, 0 to 4327951
Data columns (total 12 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   departamento_hecho      object
 1   municipio_hecho         object
 2   a_o_hechos              int32 
 3   delito                  object
 4   grupo_delito            object
 5   sexo                    object
 6   grupo_etario            object
 7   total_victimas          int64 
 8   departamento_municipio  object
 9   dept_mpio_code          object
 10  dept_name               object
 11  mupio_name              object
dtypes: int32(1), int64(1), object(10)
memory usage: 379.7+ MB


In [ ]:
# No de filas por columna que no encontraron coincidencia
print(victim_count_with_codes.isnull().sum())

departamento_hecho        0
municipio_hecho           0
a_o_hechos                0
delito                    0
grupo_delito              0
sexo                      0
grupo_etario              0
total_victimas            0
departamento_municipio    0
dept_mpio_code            0
dept_name                 0
mupio_name                0
dtype: int64


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [ ]:
# Eliminar columnas innecesarias
columns_to_drop = ['departamento_hecho', 'municipio_hecho', 'departamento_municipio', 'dept_name', 'mupio_name']
final_victim_count = victim_count_with_codes.drop(columns=columns_to_drop)

In [ ]:
# Adicionar columna para trazabilidad de la fuente
final_victim_count['source_id'] = 4

In [ ]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'a_o_hechos': 'year_of_incident',
    'delito': 'crime_type',
    'grupo_delito': 'crime_group',
    'sexo': 'sex',
    'grupo_etario': 'age_group',
    'dept_mpio_code': 'dept_mpio_code',
    'total_victimas': 'total_victims',
    'source_id': 'source_id'
}

# Renombrar las columnas
final_victim_count.rename(columns=translation_map, inplace=True)

In [ ]:
#Estructura final del dataset a integrar a la base de datos
final_victim_count.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4327952 entries, 0 to 4327951
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   year_of_incident  int32 
 1   crime_type        object
 2   crime_group       object
 3   sex               object
 4   age_group         object
 5   total_victims     int64 
 6   dept_mpio_code    object
 7   source_id         int64 
dtypes: int32(1), int64(2), object(5)
memory usage: 247.6+ MB


### Salvar en archivo csv en el drive

In [ ]:
# Guardar en archivos CSV en el drive
final_victim_count.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/victim_count.csv', index=False)